1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [11]:
import pandas as pd

In [12]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
print(ratings.head())

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [13]:
ratings_count = ratings.groupby('userId').count().reset_index()
ratings_count = ratings_count[['userId', 'rating']]
print(ratings_count.head())

   userId  rating
0       1      20
1       2      76
2       3      51
3       4     204
4       5     100


In [14]:
count_more_than_100 = ratings_count[ratings_count['rating'] > 100]
print(count_more_than_100.head())

    userId  rating
3        4     204
7        8     116
14      15    1700
16      17     363
18      19     423


In [15]:
ratings_timestamp = ratings[['userId', 'timestamp']]
rating_max = ratings_timestamp.groupby('userId').max().reset_index()
rating_min = ratings_timestamp.groupby('userId').min().reset_index()
rating_max.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)
rating_min.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)

In [16]:
print(rating_max.head())
print(rating_min.head())

   userId  timestamp_max
0       1     1260759205
1       2      835356246
2       3     1298932787
3       4      949982274
4       5     1163375145
   userId  timestamp_min
0       1     1260759108
1       2      835355395
2       3     1298861589
3       4      949778714
4       5     1163373044


In [17]:
avg_timestamp = count_more_than_100.merge(rating_max, how='inner').merge(rating_min, how='inner')
print(avg_timestamp.head())

   userId  rating  timestamp_max  timestamp_min
0       4     204      949982274      949778714
1       8     116     1154474527     1154389340
2      15    1700     1469330735      997937239
3      17     363     1127476640     1127468587
4      19     423      855195373      855190091


In [18]:
def average(row):
    avg = (row['timestamp_max'] - row['timestamp_min'])
    return avg


avg_timestamp['avg'] = avg_timestamp.apply(average, axis=1)
print(avg_timestamp.head())

   userId  rating  timestamp_max  timestamp_min        avg
0       4     204      949982274      949778714     203560
1       8     116     1154474527     1154389340      85187
2      15    1700     1469330735      997937239  471393496
3      17     363     1127476640     1127468587       8053
4      19     423      855195373      855190091       5282


2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [19]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [20]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [21]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [22]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [25]:
(client_base
 .merge(rzd, how='left', on='client_id')
 .merge(auto, how='left', on='client_id')
 .merge(air, how='left', on='client_id')
)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [26]:
table_rzd = pd.DataFrame({'client_id': [111, 112, 113, 114, 115], 'rzd_revenue': [1093, 2810, 10283, 5774, 981]})
table_auto = pd.DataFrame({'client_id': [113, 114, 115, 116, 117], 'auto_revenue': [57483, 83, 912, 4834, 98]})
table_air = pd.DataFrame({'client_id': [115, 116, 117, 118], 'air_revenue': [81, 4, 13, 173]})

In [27]:
pd.concat([table_rzd, table_auto,table_air])

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,NaN,NaN
3,114,5774.0,NaN,NaN
4,115,981.0,NaN,NaN
0,113,NaN,57483.0,NaN
1,114,NaN,83.0,NaN
2,115,NaN,912.0,NaN
3,116,NaN,4834.0,NaN
4,117,NaN,98.0,NaN


3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


Для связывания координат с визитом или фактом покупки нужно ввести ID визита, так как может быть несколько входов одного пользователя.Из координат можно получить частоту запросов из разных мест, что поможет сделать статистику покупок, предпочтений и т.д.